# Analyse d'etf - economie de guerre

# I. Importation

### Importation Librairie

In [247]:
#Importation pandas
import pandas as pd

#Importation yfinance (avec gestion d'erreur)
try:
    import yfinance as yf
    print("yfinance a été importé avec succès.")
except Exception as e:
    print(f"Erreur lors de l'importation de yfinance : {e}")

yfinance a été importé avec succès.


### Importation données (CSV)

In [248]:
df_etf = pd.read_csv("../src/analyse_portefeuille/Liste_ETF.csv", sep=";", encoding="utf-8-sig")
print(df_etf.head())

   Portefeuille _id          ISIN    Emetteur  \
0                 1  IE00BF0M2Z96         L&G   
1                 1  IE00BKLF1R75  WisdomTree   
2                 1  IE00BF4TWC33  WisdomTree   
3                 1  IE0002PG6CA6       VnEck   
4                 1  IE000U58J0M1     iShares   

                               Nom Devise Type DIST ou ACC Ticker_justetf  \
0              Battery Value-Chain    USD  ETF         ACC           BATE   
1                Battery Solutions    USD  ETF         ACC           VOLT   
2       Industrial Metals Enhanced    USD  ETF         ACC           XMWJ   
3  Rare Earth and Strategic Metals    USD  ETF         ACC           REMX   
4              Global Clean Energy    USD  ETF         ACC           INRE   

  Ticket_yf          Catergorie                    Sous_secteur  \
0    BATT.L             Énergie  Chaîne de valeur des batteries   
1    CHRG.L             Énergie              Stockage d'énergie   
2    META.L  Matières premières          

# II. Configuration

### Parametres à configurer

In [249]:
# 🎛️ Paramètres globaux de l’analyse
# devise_a_garder = 'USD'
date_debut = "2021-01-01"
date_fin = "2025-04-20"
seuil_valeurs_manquantes = 5  # En pourcentage
# colonne_ticker = 'Ticket_yf'
# colonne_nom = 'Nom'

### Initalisation tableau de données exclus

In [250]:
# Initialisation du tableau d'exclusion
df_exclus = pd.DataFrame(columns=df_etf.columns.tolist() + ['Raison'])

# III. Filtrage

### Filtrage par devise

In [251]:
#filtre des valeur EUR pour eviter problemes liée au change

#Paramètre : devise à garder
devise_a_garder = 'USD'


# Sélection des ETF avec la devise souhaitée
etf_ko_devise = df_etf[df_etf['Devise'] != devise_a_garder].copy()
etf_ko_devise['Raison'] = f"Devise ≠ {devise_a_garder}"

# Garde uniquement les ETF avec la devise correcte
df_etf = df_etf[df_etf['Devise'] == devise_a_garder]

print(f"On garde uniquement les ETF en {devise_a_garder} pour éviter les biais de change.")

On garde uniquement les ETF en USD pour éviter les biais de change.


In [252]:
# Ajout les valeurs exlus au tableau des exclus
df_exclus = pd.concat([df_exclus, etf_ko_devise], ignore_index=True)
print("Tableau des valeurs exclus : ")
print(df_exclus[["ISIN", "Nom", "Ticket_yf", "Raison"]] )

Tableau des valeurs exclus : 
           ISIN                                  Nom Ticket_yf        Raison
0  IE000FF2EBQ8  Easy ECPI Global ESG Infrastructure    ENG.PA  Devise ≠ USD
1  IE00B5MTYL84           European Food & Bev Sector   SC03.DE  Devise ≠ USD
2  IE00B5MJYX09          European Industrials Sector   SC0S.DE  Devise ≠ USD
3  IE00B5MTWH09            European Oil & Gas Sector   SC0V.DE  Devise ≠ USD


# IV. Téléchargement des donnés

In [253]:
tickers = df_etf['Ticket_yf'].dropna().unique().tolist()
df_prices = yf.download(tickers, start=date_debut, end=date_fin)['Close']

print(df_prices)

[*********************100%***********************]  12 of 12 completed

Ticker         BATT.L   CHRG.L     CSNDX.SW    CSSPX.MI      CYBP.L  DFND.AS  \
Date                                                                           
2021-01-04  16.139999  3338.25   715.700012  304.470001  519.450012      NaN   
2021-01-05  16.538000  3377.25   717.799988  304.959991  518.750000      NaN   
2021-01-06  17.054001  3474.50   719.500000  310.100006  517.799988      NaN   
2021-01-07  17.464001  3626.50   727.299988  312.769989  525.599976      NaN   
2021-01-08  17.615999  3663.00   734.900024  314.029999  536.000000      NaN   
...               ...      ...          ...         ...         ...      ...   
2025-04-11  14.628000  2007.50  1050.400024  492.820007  595.700012   5.9628   
2025-04-14  15.202000  2050.50  1078.400024  506.239990  605.450012   5.9628   
2025-04-15  15.468000  2065.75  1082.199951  510.510010  607.450012   5.9628   
2025-04-16  15.428000  2055.25  1061.199951  502.000000  601.200012   5.9628   
2025-04-17  15.358000  2039.00  1044.400

# V. Nettoyage des données de prix

In [254]:
# Pourcentage de valeurs manquantes
missing_count = df_prices.isna().sum()
missing_percent = missing_count / len(df_prices) * 100

# On écarte les ETF avec trop de données manquantes
etfs_valides = missing_percent[missing_percent < seuil_valeurs_manquantes].index
etfs_exclus = missing_percent[missing_percent >= seuil_valeurs_manquantes].index

df_etf_exclus_nan = df_etf[df_etf['Ticket_yf'].isin(etfs_exclus)].copy()
df_etf_exclus_nan['Raison'] = f"Plus de {seuil_valeurs_manquantes}% de données manquantes"
df_exclus = pd.concat([df_exclus, df_etf_exclus_nan], ignore_index=True)

# Nettoyage
df_prices = df_prices[etfs_valides]
df_prices = df_prices.ffill().bfill()

# Affichage
print("ETF écartés de l'analyse car plus de 5% de données manquantes :")
print(df_exclus)

# Aperçu du résultat
df_prices.head()

ETF écartés de l'analyse car plus de 5% de données manquantes :
  Portefeuille _id          ISIN     Emetteur  \
0                1  IE000FF2EBQ8  BNP Paribas   
1                1  IE00B5MTYL84      Invesco   
2                1  IE00B5MJYX09      Invesco   
3                1  IE00B5MTWH09      Invesco   
4                1  IE0002PG6CA6        VnEck   
5                1  IE000U58J0M1      iShares   
6                1  IE000U9ODG19      iShares   

                                   Nom Devise Type DIST ou ACC Ticker_justetf  \
0  Easy ECPI Global ESG Infrastructure    EUR  ETF         ACC            ENG   
1           European Food & Bev Sector    EUR  ETF         ACC           SC03   
2          European Industrials Sector    EUR  ETF         ACC           SC0S   
3            European Oil & Gas Sector    EUR  ETF         ACC           SC0V   
4      Rare Earth and Strategic Metals    USD  ETF         ACC           REMX   
5                  Global Clean Energy    USD  ETF       

Ticker,BATT.L,CHRG.L,CSNDX.SW,CSSPX.MI,CYBP.L,IUES.L,IWDA.L,META.L,USPY.L
Date,,,,,,,,,
2021-01-04,16.139999,3338.25,715.700012,304.470001,519.450012,3.3100,72.750000,28.125000,23.290001
2021-01-05,16.538000,3377.25,717.799988,304.959991,518.750000,3.4940,72.980003,28.450001,23.260000
2021-01-06,17.054001,3474.50,719.500000,310.100006,517.799988,3.6015,74.019997,28.680000,23.459999
2021-01-07,17.464001,3626.50,727.299988,312.769989,525.599976,3.6335,74.559998,28.940001,23.717501
2021-01-08,17.615999,3663.00,734.900024,314.029999,536.000000,3.6460,74.870003,28.620001,24.120001


# VI. Enrichissement avec Nom_complet

In [255]:
#Ajouter un nom complet
df_etf['Nom_complet'] = df_etf.apply(
    lambda row: f"{row['Emetteur']} {row['Nom']} {row['Type']} {row['DIST ou ACC']} {row['Devise']} ({row['Ticket_yf']} | {row['Ticker_justetf']}) - {row['ISIN']}",
    axis=1
)
#Leve la limite d'affichage de pendas
pd.set_option('display.max_colwidth', None)

#Verification 
df_etf[['Nom_complet', 'Nom']].head()

,Nom_complet,Nom
0,L&G Battery Value-Chain ETF ACC USD (BATT.L | BATE) - IE00BF0M2Z96,Battery Value-Chain
1,WisdomTree Battery Solutions ETF ACC USD (CHRG.L | VOLT) - IE00BKLF1R75,Battery Solutions
2,WisdomTree Industrial Metals Enhanced ETF ACC USD (META.L | XMWJ) - IE00BF4TWC33,Industrial Metals Enhanced
3,VnEck Rare Earth and Strategic Metals ETF ACC USD (REMX.L | REMX) - IE0002PG6CA6,Rare Earth and Strategic Metals
4,iShares Global Clean Energy ETF ACC USD (INRA.AS | INRE) - IE000U58J0M1,Global Clean Energy


# V Analyse

## Extraction des perf des etf sur 1 an

In [257]:
#fonction qui permet de recuperer l'historique d'un ETF entre le 01-01-2020 et 01-01-2025
def get_etf_data(ticker, start_date="2020-01-01", end_date="2025-01-01"):
    etf = yf.Ticker(ticker)
    historical_data = etf.history(start=start_date, end=end_date)
    return historical_data

#Fonction qui calcul le randement sur une periode donnée et le retourne dans une nouvelle colonne "Randement_période" (+ gestion d'erreur)
def enrichir_etf_data(row):
    ticker = row['Ticket_yf']
    try:
        historical_data = get_etf_data(ticker, start_date="2024-10-01", end_date="2025-04-01")
        rendement_6mois = (historical_data['Close'].iloc[-1] - historical_data['Close'].iloc[0]) / historical_data['Close'].iloc[0] * 100 
        row['Rendement_6mois'] = rendement_6mois
    except Exception as e:
        row['Rendement_6mois'] = None
        print(f"Erreur avec {ticker}: {e}")
    return row

# On outile les fonction pour chaque ligne (axis =1)
df_etf = df_etf.apply(enrichir_etf_data, axis=1)

# Formatage de la la colonne Rendement poru avoir 2 chiffre et "%"
df_etf['Rendement_6mois'] = df_etf['Rendement_6mois'].apply(lambda x: f"{x:.2f} %")

# Vérifier les données enrichies avec seulement les colonnes utiles
print(df_etf[['Emetteur','Nom', 'Ticket_yf', 'Rendement_6mois']])

      Emetteur                              Nom Ticket_yf Rendement_6mois
0          L&G              Battery Value-Chain    BATT.L         -9.16 %
1   WisdomTree                Battery Solutions    CHRG.L         -2.73 %
2   WisdomTree       Industrial Metals Enhanced    META.L         -6.75 %
3        VnEck  Rare Earth and Strategic Metals    REMX.L        -11.88 %
4      iShares              Global Clean Energy   INRA.AS        -21.35 %
8         Rize   Cybersecurity and Data Privacy    CYBP.L         12.32 %
9          L&G                   Cyber Security    USPY.L          6.73 %
10     iShares       Global Aerospace & Defence   DFND.AS         -1.91 %
11       VnEck                          Defense    IUES.L          5.14 %
12     iShares            S&P 500 Energy Sector    IUES.L          5.14 %
14     iShares                       Nasdaq 100  CSNDX.SW         -3.23 %
15     iShares                          S&P 500  CSSPX.MI          0.14 %
16     iShares                       M

# Exporter les données

Fusion avec le tableau df_etf

In [ ]:
# le but c'est de remplacer le ticket par le nom mais ça marche pas et faudrai concatainer.. Donc chiant. Le mieux c'est pas de concatainé ?
# et au pire sur powerbi ça pourra faire le lien auto entre les deux table non ?

ticker_to_nom = df_etf_transpose.loc['Nom'].to_dict()
df_prices.rename(columns=ticker_to_nom, inplace=True)  # optionnel

print(df_prices)

In [211]:
# Export des donnée dans un fichier CSV
data.to_csv("prix_etf.csv")